# The Notebook

## Setup

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

sns.set_palette("viridis")
sns.set_theme(style="whitegrid")

pd.set_option('display.max_colwidth', None)

## Load the data

In [ ]:
path = 'Results_HANDLEBARS_LANG3/results.csv'

Results = pd.read_csv(path, delimiter='|', names=["client", "lib", "methodName", "testName", "testResult", "testResult2ndRun", "testResultWithMutant", "testSnapshot", "testSnapshotWithMutant", "diffResultStrict", "diffResultCount", "diffResultSignature", "traceDirectoryLeft", "traceDirectoryRight", "mutationDone"])


# Remove the '/home/gus/Downloads/exps' prefix
Results['client'] = Results['client'].str.replace('/home/gus/Downloads/exps/', '', regex=False)
Results['lib'] = Results['lib'].str.replace('/home/gus/Downloads/exps/', '', regex=False)

# Filter out cases we do not want (falkiness, bad mutation, bad attach)

print("Number of runs originally:")
print(len(Results.index))

# Was able to do the mutation...
# Results = Results[Results['mutationDone'] == 'true']

# print("Number of runs after removing cases where we could not perform the mutation:")
# print(len(Results.index))

# Snapshot test is not flaky (testSnapshot == Green)
#Results = Results[Results['testSnapshot'] == 'Green']

# Only tests that are valid originally
Results = Results[Results['testResult'] == 'Green']

print("Number of runs after removing cases where the client test originally was not Green:")
print(len(Results.index))

# Client test is not flaky
#Results = Results[Results['testResult2ndRun'] == 'Green']

Results

In [ ]:
client_test_pool = Results[['client', 'lib', 'methodName', 'testName', 'testResult', 'testResult2ndRun', 'testResultWithMutant']]

print("Number of client runs originally:")
print(len(client_test_pool.index))

# Client test is not flaky
client_test_pool = client_test_pool[client_test_pool['testResult2ndRun'] == 'Green']

print("Number of client runs after removing cases where the client test is flaky:")
print(len(client_test_pool.index))

In [ ]:
gilesi_test_pool = Results[['client', 'lib', 'methodName', 'testName', 'testSnapshot', 'testSnapshotWithMutant', 'diffResultStrict', 'diffResultCount', 'diffResultSignature', 'traceDirectoryLeft', 'traceDirectoryRight', 'mutationDone']]

print("Number of gilesi runs originally:")
print(len(gilesi_test_pool.index))

# Was able to do the mutation...
gilesi_test_pool = gilesi_test_pool[gilesi_test_pool['mutationDone'] == 'true']

print("Number of gilesi runs after removing cases where we could not perform the mutation:")
print(len(gilesi_test_pool.index))

# Snapshot test is not flaky (testSnapshot == Green)
gilesi_test_pool = gilesi_test_pool[gilesi_test_pool['testSnapshot'] == 'Green']

print("Number of gilesi runs after removing cases where the snapshot test is flaky:")
print(len(gilesi_test_pool.index))

In [ ]:
ClientResultsGrouped = client_test_pool.groupby(['client', 'lib', 'methodName'])

print("Number of client mutants originally:")
print(len(ClientResultsGrouped))

GilesiResultsGrouped = gilesi_test_pool.groupby(['client', 'lib', 'methodName'])

print("Number of gilesi mutants originally:")
print(len(GilesiResultsGrouped))

mutant_list = []
mutant_metrics = []

for ClientRunsGroupName, ClientRuns in ClientResultsGrouped:
    ClientClient = ClientRunsGroupName[0]
    ClientLib = ClientRunsGroupName[1]
    ClientMethodName = ClientRunsGroupName[2]

    for GilesiRunsGroupName, GilesiRuns in GilesiResultsGrouped:
        GilesiClient = GilesiRunsGroupName[0]
        GilesiLib = GilesiRunsGroupName[1]
        GilesiMethodName = GilesiRunsGroupName[2]

        if ClientClient == GilesiClient and ClientLib == GilesiLib and ClientMethodName == GilesiMethodName:
            mutant_list.append(ClientMethodName)

            testResultWithMutant = "Green"
            testSnapshotWithMutant = "Green"

            for Run in ClientRuns["testResultWithMutant"]:
                if Run == "Red":
                    testResultWithMutant = "Red"
                    break
            
            for Run in GilesiRuns["testSnapshotWithMutant"]:
                if Run == "Red":
                    testSnapshotWithMutant = "Red"
                    break

            mutant_metrics.append([ClientClient, ClientLib, ClientMethodName, testResultWithMutant, testSnapshotWithMutant])

            break

print("Number of common mutants between both client pool and gilesi pool of tests: " + str(len(mutant_list)))

mutant_metrics = pd.DataFrame(mutant_metrics, columns=['client', 'lib', 'methodName', 'testResultWithMutant', 'testSnapshotWithMutant'])

print("Number of mutants afterwards:")
print(len(mutant_metrics.index))

#print("Loss reasons:")
#print("Lost due to client test being flaky only: " + str(drop_due_to_flaky_clienttest))
#print("Lost due to snapshot test being flaky only: " + str(drop_due_to_flaky_snapshot))
#print("Lost due to snapshot test being flaky AND client test being flaky: " + str(drop_due_to_both))

In [ ]:
mutant_list

In [ ]:
mutant_metrics

In [ ]:
overall_metrics = []

for k, el in mutant_metrics.iterrows():
    overall_metrics.append([path, el['client'], el['lib'], el['methodName'], el['testResultWithMutant'], el['testSnapshotWithMutant']])

overall_metrics = pd.DataFrame(overall_metrics, columns=['path', 'client', 'lib', 'methodName', 'testResultWithMutant', 'testSnapshotWithMutant'])

overall_metrics.to_csv('GilesiMutantDetails.csv', mode='a', index=False, header=False)

## Bucket results into 4 categories

In [ ]:
## Mutant killed by gilesi and killed by client tests
Bucket0 = mutant_metrics.query('testResultWithMutant=="Red" & testSnapshotWithMutant=="Red"')

## Mutant not killed by gilesi and not killed by client tests
Bucket1 = mutant_metrics.query('testResultWithMutant=="Green" & testSnapshotWithMutant=="Green"')

## Mutant not killed by gilesi and killed by client tests
Bucket2 = mutant_metrics.query('testResultWithMutant=="Red" & testSnapshotWithMutant=="Green"')

## Mutant killed by gilesi and not killed by client tests
Bucket3 = mutant_metrics.query('testResultWithMutant=="Green" & testSnapshotWithMutant=="Red"')

## Mutant killed by gilesi and killed by client tests

In [ ]:
Bucket0

## Mutant not killed by gilesi and not killed by client tests

In [ ]:
Bucket1

## Mutant not killed by gilesi and killed by client tests

In [ ]:
Bucket2

## Mutant killed by gilesi and not killed by client tests

In [ ]:
Bucket3

In [ ]:
# Rename Green/Red to make better sense on our matrix
mutant_metrics['testResultWithMutant'] = mutant_metrics['testResultWithMutant'].str.replace('Green', 'Not Killed', regex=False)
mutant_metrics['testResultWithMutant'] = mutant_metrics['testResultWithMutant'].str.replace('Red', 'Killed', regex=False)
mutant_metrics['testSnapshotWithMutant'] = mutant_metrics['testSnapshotWithMutant'].str.replace('Green', 'Not Killed', regex=False)
mutant_metrics['testSnapshotWithMutant'] = mutant_metrics['testSnapshotWithMutant'].str.replace('Red', 'Killed', regex=False)

# Count the occurrences of combinations between "testResultWithMutant" and "testSnapshotWithMutant"
attachment_coverage_count = mutant_metrics.groupby(['testResultWithMutant', 'testSnapshotWithMutant']).size().reset_index(name='pair_count')

# Plot the relationship
plt.figure(figsize=(8, 5))
sns.heatmap(attachment_coverage_count.pivot_table(index='testResultWithMutant', columns='testSnapshotWithMutant', values='pair_count', aggfunc='sum'),
            annot=True, fmt="f", cmap="Blues", cbar_kws={'label': 'Number of mutants'})

# Customize plot
plt.title('Mutation Score relationships')
plt.xlabel('Killed by client test')
plt.ylabel('Killed by gilesi test')
plt.tight_layout()
plt.show()